In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf

In [2]:
bathy_ds = xr.open_dataset( "/gws/nopw/j04/jmmp/jmmp_collab/AMM15/AMM15_BATHY/ADD_NICO_BALTIC_BDY_RIM_SMOOTH_BDY_" \
                        "COPY_CUTAMM15_CORRECTED_EXPANDED_MERGE_GEBCO_DEEP_TO_200-100_EMODNET_TO_10-5_" \
                        "GEBCO_TO_COAST_amm15.bathydepth.co7.cs3x.cs20.nc" ) 

In [3]:
amm15coord_ds = xr.open_dataset( "/gws/nopw/j04/jmmp/jmmp_collab/AMM15/COORDINATES/amm15.coordinates.nc" )
amm75coord_ds = xr.open_dataset( "/gws/nopw/j04/jmmp/jmmp_collab/AMM7/AMM75/INPUTS/amm75_coordinates.nc" )

In [4]:
bathy_ds = bathy_ds.rename_dims({"lat":"y", "lon":"x"})
bathy_ds = bathy_ds.assign_coords({"lat" : xr.DataArray( amm15coord_ds.gphit.values, dims=["y", "x"])} )
bathy_ds = bathy_ds.assign_coords({"lon" : xr.DataArray( amm15coord_ds.glamt.values, dims=["y", "x"])} )
bathy_ds = bathy_ds.isel(y=np.arange(1,1345),x=np.arange(1,1458))
bathy_ds = bathy_ds.assign_coords({"lat_b" : xr.DataArray( amm15coord_ds.gphif.values, dims=["y_b", "x_b"])} )
bathy_ds = bathy_ds.assign_coords({"lon_b" : xr.DataArray( amm15coord_ds.glamf.values, dims=["y_b", "x_b"])} )

In [5]:
lat_inds = np.arange(4,1345,5)
lon_inds = np.arange(2,1458,5)
amm75coord_temp_ds = amm15coord_ds.isel(lat=lat_inds, lon=lon_inds)

amm75bathy_ds = xr.Dataset()
amm75bathy_ds = amm75bathy_ds.assign_coords({"lat" : xr.DataArray( amm75coord_temp_ds.gphit.values, dims=["y", "x"])} )
amm75bathy_ds = amm75bathy_ds.assign_coords({"lon" : xr.DataArray( amm75coord_temp_ds.glamt.values, dims=["y", "x"])} )
amm75bathy_ds = amm75bathy_ds.isel(y=np.arange(1,269),x=np.arange(1,292))
amm75bathy_ds = amm75bathy_ds.assign_coords({"lat_b" : xr.DataArray( amm75coord_temp_ds.gphif.values, dims=["y_b", "x_b"])} )
amm75bathy_ds = amm75bathy_ds.assign_coords({"lon_b" : xr.DataArray( amm75coord_temp_ds.glamf.values, dims=["y_b", "x_b"])} )



In [167]:
regrid = xesmf.Regridder( bathy_ds, amm75bathy_ds, method="conservative" )

In [168]:
amm75bathy05 = regrid( bathy_ds.Bathymetry, skipna=True, na_thres=0.5 )[:-1,:-1]

In [169]:
amm75bathy05

<xarray.DataArray (y: 267, x: 290)>
array([[4302.45166017, 4383.86279297, 3988.00781249, ...,           nan,
                  nan,           nan],
       [4518.52246088, 4512.65039063, 4430.21337891, ...,           nan,
                  nan,           nan],
       [4508.16552739, 4517.89892578, 4529.58398445, ...,           nan,
                  nan,           nan],
       ...,
       [1619.31188965, 1657.19592287, 1736.86022953, ...,           nan,
                  nan,           nan],
       [1534.98730469, 1585.79870602, 1614.00708003, ...,           nan,
                  nan,           nan],
       [1530.07434082, 1486.52416991, 1522.01586914, ...,           nan,
                  nan,           nan]])
Coordinates:
    lon      (y, x) float32 -17.52 -17.43 -17.34 -17.25 ... 15.8 15.94 16.08
    lat      (y, x) float32 44.2 44.22 44.23 44.24 ... 62.21 62.19 62.18 62.16
Dimensions without coordinates: y, x
Attributes:
    regrid_method:  conservative

**Now manually remove lakes, isolated points, some coastal smoothing and open up channels to Baltic**

In [6]:
amm75bathy05_ds = xr.Dataset()
amm75bathy05_ds["bathy"] = amm75bathy05
amm75bathy05_ds.to_netcdf("../PROCESSED_DATA/amm75_bathy05_coastalsmoothed_all_closeseas_removed_and_baltic_open_final.nc")
